# Load Modules

In [1]:
#pip install scikit-learn

In [2]:
#pip install --upgrade seaborn

In [3]:
#pip install npTDMS

In [4]:
#import seaborn as sns

In [5]:
import os
import re
import datetime
import requests
import json
import pandas
import scrapbook
import numpy as np
from nxtdms import TdmsFile
import pandas as pd
import random
import matplotlib.pyplot as plt
import pickle
import scrapbook as sb
from scipy.interpolate import CubicSpline,PchipInterpolator
from requests.exceptions import HTTPError
import sys
!{sys.executable} -m pip install --upgrade hvplot | grep -v 'already satisfied'
import json

import warnings
warnings.filterwarnings('ignore')

from systemlink.clients.nifile.api.files_api import FilesApi
from systemlink.clients.nitestmonitor import ResultsApi
from systemlink.clients.nitestmonitor.models.update_test_results_request import UpdateTestResultsRequest
from systemlink.clients.nifile.models.property_query import PropertyQuery
from systemlink.clients.nifile.models.query_available_files_request import QueryAvailableFilesRequest

import holoviews as hv
import hvplot.pandas
hv.extension('bokeh')

api_key = os.getenv("SYSTEMLINK_API_KEY")
sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}

  Using cached hvplot-0.12.1-py3-none-any.whl.metadata (19 kB)
Using cached hvplot-0.12.1-py3-none-any.whl (175 kB)
  Attempting uninstall: hvplot
    Found existing installation: hvplot 0.11.2
    Uninstalling hvplot-0.11.2:
      Successfully uninstalled hvplot-0.11.2


# Settings

In [6]:
api_key

'jl7WnPk8UM5a8TNBX52u3q4n85PaUH_zVzi7UObqHq'

In [7]:
sl_uri

'https://test-api.lifecyclesolutions.ni.com'

## Parameters

In [8]:
#parameters

## Using Default values when None were given by Automatic Run

In [9]:
try:
    files_api = FilesApi()
    results_api = ResultsApi()
    bRunInSLE=1
except:
    bRunInSLE=0


In [10]:
 # EMEA ESW Internal 71095276-6ffb-4a55-8fe3-ad37943925d8  EMEA Cust 2b075130-bbd0-40cf-b0a8-ecea1c0d5f81
filename_ErrOut=r'PreviousFilesErrCols_Running.pkl'
filename_ErrIn=filename_ErrOut
myWorkspace = "Data Science"
bDebug=0
try:
    file_ids
except NameError:
    bDebug=1
    workspace = '2b075130-bbd0-40cf-b0a8-ecea1c0d5f81'
    file_ids = ['1ace620c-79aa-46c7-b0f4-2e12ea48e15a']#][#'19c7e045-fd2a-42d1-9dce-a75f5dd4a65b']# ["1689bcc4-e010-47dd-900b-e42a44da882c"]#
    #filename=r'./comp_HFSM9422102PEN_138862B-01L_335B813_04-01-2024_15-59-14-514.tdms'
    filename_ErrIn=r'PreviousFilesErrCols.pkl'


N_SEED=42000
N=5 # Interpolation length
b_use_ml=0
path_file_inout=r'./Data_IO/'
z_threshold=3.
bApplyUpperLimit=1
bApplyLowerLimit=0

bUpdateMetaData=1

## Select the Workspace to create the data table

In [11]:
if bRunInSLE==1:
    headers = { 'X-NI-API-KEY': api_key }
    get_workspaces_resp = requests.get(f'{sl_uri}/niuser/v1/workspaces', headers=headers)
    
    # Expect a 200 code on success, raise an exception if there is an error response
    get_workspaces_resp.raise_for_status()
    
    workspaces = get_workspaces_resp.json()["workspaces"]
    
    for i in workspaces:
        if i["name"] == myWorkspace:
            workspace = i 

# Functions

## SLE

### SLE File Handling

In [12]:
#import numpy as np
#import pandas as pd
#import json
#from datetime import datetime

def transform_dict(input_dict):
    transformed_dict = {}
    for key, value in input_dict.items():
        # Transform key
        if isinstance(key, float):
            new_key = float(key)
        elif np.issubdtype(type(key), np.integer):
            new_key = int(key)
        elif isinstance(key, np.bool_):
            new_key = bool(key)
        elif np.issubdtype(type(key), np.datetime64):
            new_key = pd.to_datetime(key).isoformat()
        else:
            new_key = key
        
        # Transform value
        if isinstance(value, float):
            new_value = float(value)
        elif np.issubdtype(type(value), np.integer):
            new_value = int(value)
        elif isinstance(value, np.bool_):
            new_value = bool(value)
        elif np.issubdtype(type(value), np.datetime64):
            new_value = pd.to_datetime(value).isoformat()
        else:
            new_value = value
        
        transformed_dict[new_key] = new_value
    
    return transformed_dict


In [13]:
def GetFileByID(file_id,path_out=r'./',sl_uri=None):
    if sl_uri is None: 
        sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
    try:
        download_resp = requests.get(f'{sl_uri}/nifile/v1/service-groups/Default/files/{file_id}/data', headers=headers)
        download_resp.raise_for_status()
    except HTTPError as exc:
        print("File with id " + file_id + " not found")
        raise
    filename_out = download_resp.headers['content-disposition'].split('"')[1::-1][0]
    filename_out=os.path.join(path_out,filename_out)
    with open(filename_out, 'wb') as file:
        file.write(download_resp.content)
    return filename_out
    

In [14]:
async def GetLatestFile(filename_in,path_out=r'./',sl_uri=None):
    if sl_uri is None: 
        sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
    property_query = PropertyQuery(key="Name", operation="EQUAL", value=filename_in)
    query = QueryAvailableFilesRequest(properties_query=[property_query])
    res = await files_api.query_available_files(query=query)
    i=0
    max_dt=None
    i_max=0
    id_max=None
    for dic in res.available_files:
        if max_dt is None or max_dt<dic.created:
            i_max=i
            id_max=dic.id  
            max_dt=dic.created
            #print(i_max)
        i+=1
    file_id=res.available_files[i_max].id
    filename_out=GetFileByID(file_id,path_out,sl_uri)
    return file_id,filename_out
        
    

In [15]:
def generate_column_specs_from_dataframe(df):
    columns_specs = []
    column_types = []
    
    # Convert object columns to strings
    df = df.astype({col: str for col in df.select_dtypes(include=['object']).columns})
    
    for column in df.columns:
        if str(df[column].dtypes) == 'float64':
            column_types.append('FLOAT64')
        elif str(df[column].dtypes) == 'float32':
            column_types.append('FLOAT32')    
        elif 'datetime64' in str(df[column].dtypes):  # attention 
            column_types.append('TIMESTAMP')
        elif str(df[column].dtypes) == 'object':
            column_types.append('STRING')
        elif str(df[column].dtypes) == 'int64':
            column_types.append('INT64')
        elif str(df[column].dtypes) in ['uint8', 'int16', 'int32']:
            column_types.append('INT32')
    
    for column_name, dataType in zip(df.columns, column_types):
        column_type = 'INDEX' if column_name == 'index' else 'NULLABLE'
        columns_specs.append({
            'name': column_name, 
            'dataType': dataType, 
            'columnType': column_type, 
            'properties': {}
        })
    
    return columns_specs

### SLE DataFrame

In [16]:
def CreateSLDataFrame(dfIn,workspace_id,table_name=None,sl_uri=None,api_key=None):
    if sl_uri==None:
        sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
    if api_key==None: 
        api_key = os.getenv("SYSTEMLINK_API_KEY")
        
    columns=generate_column_specs_from_dataframe(dfIn)
    # Create the table
    table_info = {'name': table_name, 'columns': columns, 'workspace': workspace_id}
    headers = { "X-NI-API-KEY": api_key, 'Content-Type': 'application/json' }
    create_table_resp = requests.post(f'{sl_uri}/nidataframe/v1/tables', headers=headers, json=table_info)
    
    # Expect 201 on success, raise an exception if there is an error response
    create_table_resp.raise_for_status()
    
    table_id = create_table_resp.json()["id"]
    return table_id



In [17]:
def UpdateSLDataFrame(dfIn,table_id,sl_uri=None,api_key=None):
    if sl_uri==None:
        sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
    if api_key==None: 
        api_key = os.getenv("SYSTEMLINK_API_KEY")
        
    # Convert the pandas dataframe to the expected json format
    frame_dict = dfIn.astype(str).to_json(orient="split", index=False) #.astype(str)
    
    # Write the data to the table. Since we're writing one file/frame per table, set endofData to true
    frame_info = f'{{"frame": {frame_dict}, "endOfData": false}}'
    headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}
    write_data_resp = requests.post(f'{sl_uri}/nidataframe/v1/tables/{table_id}/data', headers=headers, data=frame_info)
    
    # Expect 204 on success, raise an exception if there is an error response
    return write_data_resp.raise_for_status()

In [18]:
def CreateAndUpdateSLDataFrame(dfIn,workspace_id,table_name=None,sl_uri=None,api_key=None):
    if 'index' not in dfIn.columns:
        dfIn=dfIn.reset_index()
    if sl_uri==None:
        sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
    if api_key==None: 
        api_key = os.getenv("SYSTEMLINK_API_KEY")
    if table_name is None:
        raise TypeError
    table_id=CreateSLDataFrame(dfIn,workspace_id,table_name=table_name,sl_uri=sl_uri,api_key=api_key)
    response_status=UpdateSLDataFrame(dfIn,table_id,sl_uri,api_key)
    return table_id, response_status
    
    

In [19]:
def QueryDataFrame(table_id,sl_uri,json_body,continuationToken=None,verbose=0):
    if continuationToken is not None: 
        json_body.update({"continuationToken":continuationToken})
    response=requests.post(f'{sl_uri}/nidataframe/v1/tables/{table_id}/query-data', headers=headers, data=json.dumps(json_body))
    if response.status_code == 200 or response.status_code == 201:
        #print("Okay")
        totalRowCount=json.loads(response.text)['totalRowCount']
        continuationToken=json.loads(response.text)['continuationToken']
        if verbose==1:
            print(f"Succesfull Query: Total Rows={totalRowCount}, continuationToken={continuationToken}")
        #print("Response Body: " + json.dumps(json.loads(response.text), indent=4))
    #if continuationToken is None: break
    else: response.raise_for_status()
    dfOut=pd.DataFrame(data=json.loads(response.text)['frame']['data'],columns=json.loads(response.text)['frame']['columns'])
    totalRowCount=json.loads(response.text)['totalRowCount']
    continuationToken=json.loads(response.text)['continuationToken']
    return dfOut, continuationToken, totalRowCount
    

## TDMS

In [20]:
def parse_file_name(file_name): # parse Product and UUID from file name
    parts = file_name.split('_')
    product = parts[2]
    uuid = parts[3]
    return product, uuid

def read_tdms_file_properties(file_path): # parse Status, Operation and Start time from TDMS header properties
    start_time = time.time()
    try:
        with TdmsFile.read(file_path) as tdms_file:
            hwt_status = tdms_file.properties.get('HWT_Status', None)
            hwt_op_sequence = tdms_file.properties.get('HWT_OpSequence', None)
            hwt_procedure_start_time = tdms_file.properties.get('HWT_ProcedureStartTime', None)
            
            return {
                'Status': hwt_status,
                'Operation': hwt_op_sequence,
                'Start_time': hwt_procedure_start_time
            }, time.time() - start_time
    except Exception as e:
        print(f"Error reading properties from file: {file_path}. Error: {e}")
        return None, time.time() - start_time

In [21]:
def extract_reading_data_from_file(file_path,read_groups=None): # Extracts data from the 'Reading' channels of a TDMS file
    data_df = pd.DataFrame(columns=['File_Name', 'Group', 'Index_wf', 'Reading']) # Create an empty DataFrame to store the data
    if os.path.exists(file_path): # Read TDMS file and extract data from the "Reading" channel if it exists
        with TdmsFile.open(file_path) as tdms_file:
            for group in tdms_file.groups(): # iterate over the different groups in the tdms file
                if read_groups is not None:
                    if group.name not in read_groups:
                        continue
                for channel in group.channels(): # iterate over the different channels in each group
                    if channel.name == "Reading":
                        data = pd.DataFrame({
                            'File_Name': [os.path.basename(file_path)] * len(channel[:]),
                            'Group': [group.name] * len(channel[:]),
                            'Index_wf': range(len(channel[:])),
                            'Reading': channel[:]
                        })
                        data_df = pd.concat([data_df, data], ignore_index=True)
    else:
        print(f"File '{file_path}' not found.")

    return data_df

def extract_reading_all_channels_from_file(file_path,read_groups=None): # Extracts data from the 'Reading' channels of a TDMS file
    #data_df = pd.DataFrame(columns=['File_Name', 'Group', 'Index_wf', 'Reading']) # Create an empty DataFrame to store the data
    if os.path.exists(file_path):
        ldata=[] # Read TDMS file and extract data from the "Reading" channel if it exists
        with TdmsFile.open(file_path) as tdms_file:
            for group in tdms_file.groups(): # iterate over the different groups in the tdms file
                if read_groups is not None:
                    if group.name not in read_groups:
                        continue
                for channel in group.channels(): # iterate over the different channels in each group
                    #if channel.name == "Reading":
                    data = pd.DataFrame({
                        'File_Name': [os.path.basename(file_path)] * len(channel[:]),
                        'TestSuiteName': [group.name] * len(channel[:]),
                        'TestName': [channel.name] * len(channel[:]),
                        'Index_wf': range(len(channel[:])),
                        'FloatValue': channel[:]
                    })
                    ldata.append(data)
                       
    else:
        print(f"File '{file_path}' not found.")
    data_df = pd.concat(ldata, ignore_index=True) 
    return data_df

def extract_data_from_folder_for_files(folder_path, file_list): # Extracts data from TDMS files within a folder and its subfolders for files with names in the specified list
    all_data_df = pd.DataFrame() # Create an empty DataFrame to store data from all files   
    for root, dirs, files in os.walk(folder_path, topdown=True): # Search for TDMS files in the folder and its subfolders
        for file_name in files:
            for file in file_list:
                if file == file_name:
                    file_path = os.path.join(root, file_name)
                    file_data_df = extract_reading_data_from_file(file_path)
                    all_data_df = pd.concat([all_data_df, file_data_df], ignore_index=True)

    return all_data_df




In [22]:
def CreateSweepCounter(dfIn,xname='Frequency'):
    df_freq=dfIn[dfIn['TestName']==xname].copy()
    df_freq['Condition']=np.where(df_freq['FloatValue']<df_freq['FloatValue'].shift(),1,0).cumsum()
    df_out=dfIn.merge(df_freq[['Condition','Index_wf']],on='Index_wf').copy()
    return df_out

def CreateSweepCounter_pivot(dfIn,xname='frequency'):
    dfOut=dfIn.sort_values('RepeatitionOrder').copy()
    dfOut['Condition']=np.where(dfOut[xname]<dfOut[xname].shift(),1,0).cumsum()
    return dfOut

def Get_Waveform(dfIn,xname='RepeatitionOrder',yname='Reading'):
    y=dfIn.sort_values(xname)[yname].values
    return y

In [23]:
def CreateMetaData(dfIn,xname='File_Name'):
    lmeta=[]
    lmeta+=['Station','Product','UUID']
    dfIn[lmeta]=dfIn[xname].str.split('_',expand=True)[[1,2,3]]
    return dfIn,lmeta

def ExtractDateTime(dfIn,xname='File_Name',datetime_pattern=re.compile(r'(\d{2}-\d{2}-\d{4}_\d{2}-\d{2}-\d{2})')):
    # Extract the datetime string
    match = datetime_pattern.search(dfIn[xname])
    if match:
        datetime_str = match.group(1)
        # Parse the datetime string into a datetime object
        dt = datetime.datetime.strptime(datetime_str, '%m-%d-%Y_%H-%M-%S')
        #print(f"Extracted datetime: {dt}")
    else:
        print("No datetime found in the filename.")
    return dt



## Anomaly Score

In [24]:
def CreateErrorMetrics(dfIn,yname_true='y',yname_pred='y_pred',idx_control=[],yout_suffix=''):
    Y_True=np.stack(dfIn[yname_true])
    Y_Pred=np.stack(dfIn[yname_pred])
    Y_Delta=Y_True-Y_Pred
    if len(idx_control)>0:
        #print('Control')
        #Y_Delta_control=Y_train[:,control_idx]-Y_Pred_Train[:,control_idx]
        Y_Delta_control=Y_Delta[:,idx_control]
        #print(Y_Delta_control.shape)
        MAE_control=np.mean(np.abs(Y_Delta_control),axis=1)
        dfIn['Error_'+'MAE'+yout_suffix]=MAE_control
        MSE_control=np.mean(Y_Delta_control**2,axis=1)
        dfIn['Error_'+'MSE'+yout_suffix]=MSE_control
        MAPE_control=np.mean(np.abs(Y_Delta_control/Y_True[:,idx_control]),axis=1)
        dfIn['Error_'+'MAPE'+yout_suffix]=MAPE_control
        dfIn['Error_'+'RMSE'+yout_suffix]=np.sqrt(MSE_control)
    else:
        MAE=np.mean(np.abs(Y_Delta),axis=1)
        dfIn['Error_'+'MAE'+yout_suffix]=MAE
        MSE=np.mean(Y_Delta**2,axis=1)
        dfIn['Error_'+'MSE'+yout_suffix]=MSE
        MAPE=np.mean(np.abs(Y_Delta/Y_True),axis=1)
        dfIn['Error_'+'MAPE'+yout_suffix]=MAPE
        dfIn['Error_'+'RMSE'+yout_suffix]=np.sqrt(MSE)
    return dfIn

In [25]:
def Get_Interpolations(dfIn_,X,cl_models={'linear':lambda xp,yp,x: np.interp(x,xp,yp),'cubic':lambda xp,yp,x: CubicSpline(xp,yp)(x),'pchip':lambda xp,yp,x: PchipInterpolator(xp,yp)(x)},idx_control=None):#,ml_models=None,user_models=None,):
    #dfIn_=dfIn_.sort_values('xp')
    yp=dfIn_['yp']
    xp=dfIn_['xp']
    results={}
    for key, func in cl_models.items():
        #print(key)
        result=func(xp,yp,X)
        #print(result.shape)
        dfIn_[f'y_inter_{key}']=result
        if idx_control is not None:
            result_control=result[idx_control]
            dfIn_[f'y_control_{key}']=result_control
 
    return dfIn_



#dft

In [26]:
def apply_limits(dfIn, xname='value', bApplyUpperLimit=1, bApplyLowerLimit=1,PassLimitDefault=np.nan,PassUSLDefault=np.nan,PassLSLDefault=np.nan):
    dfOut = dfIn.copy()
    
    # Initialize the new columns
    dfOut['PassLimit'] = PassLimitDefault
    dfOut['PassUSL'] = PassUSLDefault
    dfOut['PassLSL'] = PassLSLDefault

    for index, row in dfOut.iterrows():
        # Apply Upper Limit
        if bApplyUpperLimit == 1:
            if not pd.isna(row['USL']):
                if row[xname] > row['USL']:
                    #dfOut.at[index, 'PassLimit'] = 0
                    dfOut.at[index, 'PassUSL'] = 0
                else:
                    dfOut.at[index, 'PassUSL'] = 1
            #print(f"USL {row['USL']:.2f}")
        
        # Apply Lower Limit
        if bApplyLowerLimit == 1:
            if not pd.isna(row['LSL']):
                if row[xname] < row['LSL']:
                    #dfOut.at[index, 'PassLimit'] = 0
                    dfOut.at[index, 'PassLSL'] = 0
                else:
                    dfOut.at[index, 'PassLSL'] = 1
            #print(f"LSL {row['LSL']:.2f}")
        if dfOut.at[index, 'PassUSL'] == 0 or dfOut.at[index, 'PassLSL'] == 0:
            dfOut.at[index, 'PassLimit'] = 0
        else:
            dfOut.at[index, 'PassLimit'] = 1
        if pd.isna(dfOut.at[index, 'PassUSL'])  and pd.isna(dfOut.at[index, 'PassLSL']):
            dfOut.at[index, 'PassLimit'] = np.nan
    
    return dfOut

In [27]:
def update_statistics(N_old, mean_old, var_old, N_new, mean_new, var_new):
    # Update mean
    mean_updated = (N_old * mean_old + N_new * mean_new) / (N_old + N_new)
    
    # Update variance
    var_updated = (
        ((N_old - 1) * var_old + (N_new - 1) * var_new + 
         (N_old * N_new / (N_old + N_new)) * (mean_old - mean_new)**2) / 
        (N_old + N_new - 1)
    )
    return mean_updated, var_updated

# ETL on Single DUT Data

## Extract

### Load Data

In [28]:
if len(file_ids) != 1:
    raise Exception(f"This example notebook is designed to operate on one file at a time. Length is {len(file_ids)}")
os.makedirs(path_file_inout, exist_ok=True)
if bRunInSLE==1:
    filename_in=GetFileByID(file_ids[0],path_out=path_file_inout)
else:
    filename_in=os.path.join(path_file_inout,filename)
filename=os.path.split(filename_in)[-1]
print(filename_in)
df_org=extract_reading_all_channels_from_file(filename_in,read_groups=['TX_RETURN_LOSS(dB)'])#,'RX_RETURN_LOSS(dB)'])

./Data_IO/comp_HFSM9422101PEN_138862A-01L_32A1319_08-06-2023_20-05-39-176.tdms


In [29]:
os.path.split(filename_in)[-1]

'comp_HFSM9422101PEN_138862A-01L_32A1319_08-06-2023_20-05-39-176.tdms'

In [30]:
print(file_ids[0])

1ace620c-79aa-46c7-b0f4-2e12ea48e15a


In [31]:
#!pip list

In [32]:
workspace

{'default': False,
 'enabled': True,
 'id': 'ee940aa2-05d3-4585-a822-52f7234a5207',
 'name': 'Data Science'}

### Update MetaData

In [33]:
if bUpdateMetaData==1:
    tdms_probs= TdmsFile.open(filename_in).properties
    dic_tdms_probs=dict(tdms_probs)
    tr_dict=transform_dict(dic_tdms_probs)
    dicMeta={  "replaceExisting": True,
              "expectedRevision": 0,
              "workspace": workspace['id']
            }
    meta_prop=pd.Series(dict(tr_dict)).astype(str).to_frame(name='properties').to_dict()
    dicMeta.update(meta_prop)
    #def UpdateMetaData(dfIn,table_id,sl_uri=None,api_key=None):
    if sl_uri==None:
        sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")
    if api_key==None: 
        api_key = os.getenv("SYSTEMLINK_API_KEY")
        
    # Convert the pandas dataframe to the expected json format
    frame_dict = json.dumps(dicMeta)#dfIn.astype(str).to_json(orient="split", index=False) #.astype(str)
    table_id=file_ids[0]
    # Write the data to the table. Since we're writing one file/frame per table, set endofData to true
    frame_info = frame_dict#f'{{"frame": {frame_dict}, "endOfData": false}}'
    headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}
    write_data_resp = requests.post(f'{sl_uri}/nifile/v1/service-groups/Default/files/{table_id}/update-metadata',headers=headers, data=frame_info)
    
    # Expect 204 on success, raise an exception if there is an error response
    write_data_resp.raise_for_status()

In [34]:
#await files_api.upload(file=filename_in,workspace=workspace,metadata=tr_dict)

### Load Model

In [35]:
if b_use_ml==1:
    base_model = RandomForestRegressor(n_estimators=100, random_state=42)
    # Turn it into a multi-output regressor
    model = MultiOutputRegressor(base_model)
    
    sFolder=r'./Models/ML_Interpolation/'
    sModel_base=r'TX_Return__Interp_5_20240725135229'#r'TX_Return__Interp_5_20240717164308' './Models/ML_Interpolation/Model_TX_Return__Interp_5_20240725135229.pkl'
    sModel=f"{sFolder}Model_{sModel_base}.pkl"
    sdf_y_train=f"{sFolder}df_y_train_{sModel_base}.parquet"
    sdf_y_test=f"{sFolder}df_y_test_{sModel_base}.parquet"
    N=int(sModel.split('Interp_')[1].split('_')[0])
    print(sModel)
    print(sdf_y_train)
    print(sdf_y_test)
    model=pickle.load(open(sModel, 'rb'))
    
    print(N)


## Transform

### Data Preprocessing

In [36]:
dfIn=df_org.copy()#df_org[df_org['TestSuiteName']=='TX_RETURN_LOSS(dB)'].copy()

In [37]:
dfIn[['Station','Product','UUID']]=dfIn['File_Name'].str.split('_',expand=True)[[1,2,3]]

In [38]:
#dfIn_freq=dfIn[dfIn['TestName']=='Frequency'].copy()
#dfIn_freq['SweepCounter']=np.where(dfIn_freq['FloatValue']<dfIn_freq['FloatValue'].shift(),1,0).cumsum()

In [39]:
dfIn_freq=dfIn.groupby(['File_Name','TestSuiteName']).apply(CreateSweepCounter)#.reset_index()#,include_groups=False).reset_index()

In [40]:
dfIn2=dfIn.merge(dfIn_freq.reset_index(drop=True)[['File_Name','TestSuiteName','TestName','Condition','Index_wf']],on=['File_Name','TestName','TestSuiteName','Index_wf'],how='left').copy()

In [41]:
dfIn2=dfIn2.rename(columns={'SweepCounter':'Condition'})

In [42]:
dfIn2_ConditionSettings=dfIn2.groupby(['File_Name','Condition','TestSuiteName','TestName']).nunique().reset_index().groupby('TestName').max()

In [43]:
lConditionSettings=list(dfIn2_ConditionSettings[dfIn2_ConditionSettings['FloatValue']==1].index)

In [44]:
#lConditionSettings

In [45]:
dfOutMeta=dfIn2[dfIn2['TestName'].isin(lConditionSettings)].copy()

In [46]:
dfOut=dfIn2.groupby(['File_Name','TestSuiteName','TestName','Condition']).apply(lambda x: Get_Waveform(x,xname='Index_wf',yname='FloatValue')).reset_index()#.reset_index(drop=True)#,include_groups=False).reset_index()#.rename_axis(columns={0,'y'})
dfOut=dfOut[~dfOut['TestName'].isin(lConditionSettings)].reset_index(drop=True)
dfOut=dfOut.rename(columns={0:'y'})
#dfOut.head()

In [47]:
dfOut[['Station','Product','UUID']]=dfOut['File_Name'].str.split('_',expand=True)[[1,2,3]]
dfOut['StartTime']=dfOut.apply(ExtractDateTime,axis=1)

In [48]:
dfOut2=dfOut.pivot(index=['Station','Product','UUID','StartTime','File_Name','TestSuiteName','Condition'],columns='TestName',values='y')#.reset_index()dfIn[['Station','Product','UUID']]=dfIn['File_Name'].str.split('_',expand=True)[[1,2,3]]

In [49]:
dfOut2['y_org']=dfOut2['Reading'].copy()
dfOut2['x_org']=dfOut2['Frequency'].copy()
freqs=dfOut2.iloc[0]['x_org']

In [50]:
df_sampling=pd.Series(np.zeros(len(freqs)),index=freqs).to_frame('Sampling').reset_index().rename(columns={'index':'Frequency'})
df_sampling_model=df_sampling[df_sampling.index%N==0]
df_sampling_nomodel=df_sampling[df_sampling.index%N!=0]
control_percent=0.05
N_control=int(np.ceil(len(freqs)*control_percent))
df_sampling_control=df_sampling_nomodel.sample(n=N_control,random_state=N_SEED,replace=False).sort_values('Frequency')
control_idx=list(df_sampling_control.index)
sample_idx=df_sampling_model.index


### Apply Model

In [51]:
dfOut2['yp']=dfOut2['y_org'].apply(lambda x: x[sample_idx])
dfOut2['y_control']=dfOut2['y_org'].apply(lambda x: x[control_idx])

dfOut2['xp']=dfOut2['x_org'].apply(lambda x: x[sample_idx])
dfOut2['x_control']=dfOut2['x_org'].apply(lambda x: x[control_idx])
dft=dfOut2.copy()
dft=dft.apply(lambda x: Get_Interpolations(x,freqs,idx_control=control_idx),axis=1)

In [52]:
#ml_models={'mlrf':lambda xp,yp,x:model.predict(yp.reshape(1,-1))})
if b_use_ml==1:
    dic_mlrf={'name':'mlrf',
                'model':model,
                'ApplyOn':{'TestSuiteName':'TX_RETURN_LOSS(dB)','Product':'138862B-01L'},            
                }
    sname=dic_mlrf['name']
    print(sname)
    df_model=dft.reset_index()
    df_model[['Station','Product','UUID']]=df_model['File_Name'].str.split('_',expand=True)[[1,2,3]]
    for key, value in dic_mlrf['ApplyOn'].items():
        print(f"{key}:{value}")
        df_model=df_model[df_model[key]==value]
        Xin=np.stack(df_model['yp'])
        result=dic_mlrf['model'].predict(Xin)
        df_model[f'y_inter_{sname}']=list(result)
        if control_idx is not None:
            result_control=list(result[:,control_idx])
            df_model[f'y_control_{key}']=result_control       


In [53]:
lindex=list(dft.index.names)

### Calculate Anomaly Score

In [54]:
dft2=dft.copy()
for col in dft.filter(like='y_control_').columns:
    dft2=CreateErrorMetrics(dft2,yname_true='y_control',yname_pred=col,yout_suffix='_'+col.split('_',1)[-1])

## Load

### Update Anomaly Score Table

### Append Data

In [55]:
ErrCols=dft2.filter(like='Error_').columns
dft3=dft2[ErrCols].copy()

In [56]:
if bRunInSLE==1:
    file_id_,filename_=await GetLatestFile(filename_ErrIn,path_out=path_file_inout)
else:
    filename_=os.path.join(path_file_inout,filename_ErrIn)
dfErr=pd.read_pickle(filename_)
#dfErr.head()

IndexError: list index out of range

In [57]:
filename_ErrIn

'PreviousFilesErrCols.pkl'

In [ ]:
dfErrOut=pd.concat([dfErr.reset_index(),dft3.reset_index()])
filtered_columns = [col for col in dfErrOut.columns if not col.startswith('Error')]
dfErrOut=dfErrOut.set_index(filtered_columns)
duplicates = dfErrOut.index.duplicated()
dfErrOut=dfErrOut[~duplicates]
dfErrOut.to_pickle(filename_ErrOut)
if bRunInSLE==1:
    await files_api.upload(file=filename_ErrOut)
    

In [ ]:
if bDebug==1:
    dfErr=dfErr.set_index('StartTime',append=True)

In [ ]:
dfErrStats=dfErr.melt(value_vars=ErrCols,ignore_index=False).groupby(['Product','TestSuiteName']+['variable'])['value'].agg([len,'mean','std']).reset_index()
dfErrStats
if bApplyUpperLimit==1:
    dfErrStats['USL']=dfErrStats['mean']+z_threshold*dfErrStats['std']
if bApplyLowerLimit==1:
    dfErrStats['LSL']=dfErrStats['mean']-z_threshold*dfErrStats['std']
dfErrStats=dfErrStats.rename(columns={'len':'N'})
dfErrStats

In [ ]:
#dfErr.melt(value_vars=ErrCols,ignore_index=False)#,ignore_index=False).reset_index()

In [ ]:
#dfErr_m=pd.merge(dfErr.melt(value_vars=ErrCols,ignore_index=False).reset_index(),dfErrStats, on=['Product','TestSuiteName','variable'],how='left')
dfErr_m=pd.merge(dfErrOut.melt(value_vars=ErrCols,ignore_index=False).reset_index(),dfErrStats, on=['Product','TestSuiteName','variable'],how='left')
#dfErrOut

In [ ]:
dfErr_m

In [ ]:
dfErrOut2=apply_limits(dfErr_m.copy(), bApplyUpperLimit=1, bApplyLowerLimit=0)

In [ ]:
dfErrOut2

In [ ]:
filename_out=os.path.split(filename_in)[-1]

In [ ]:
TdmsFile.open(filename_in).properties

In [ ]:
for key, value in  tdms_probs.items():
        #df_out[key]=value
        print(key, value)

In [ ]:
dict(tdms_probs)

In [ ]:
#json.dumps(dict(tdms_probs))

In [ ]:
dfErrOut2[dfErrOut2['File_Name']==filename_out]

In [ ]:
dft3

In [ ]:
dfErrOut2_pivot = dfErrOut2.pivot_table(index=['UUID', 'Station', 'Product', 'File_Name', 'TestSuiteName', 'Condition',
       'StartTime'], columns='variable', values=['value', 'USL', 'PassLimit', 'PassUSL'])
dfErrOut2_pivot.columns = [f'{col[1]}_{col[0]}' for col in dfErrOut2_pivot.columns]
dfErrOut2_pivot=dfErrOut2_pivot.reset_index()

In [ ]:
dfErrOut2_pivot

In [ ]:
dfErrOut2_pivot[dfErrOut2_pivot['File_Name']==filename_out]

In [ ]:
dfErr_Result=dfErr.reset_index()

### Define Output of Jupyter notebook

In [ ]:
if bRunInSLE==1:
    result = [
        {
            'id': 'error_info',
            'type': 'data_frame',
            'data': {'columns': pd.io.json.build_table_schema(dfErr_Result, index=False)['fields'], 'values': dfErr_Result.values.tolist()}
        }
    ]
    sb.glue('result', result)

In [ ]:
dfErrOut2_pivot[dfErrOut2_pivot['UUID']=='335B813']

### Write pandas dataframe into SystemLink DataFrame 

In [ ]:
columns=generate_column_specs_from_dataframe(dfErrOut2.reset_index())
table_name='AnomalyScore'
# Create the table
table_info = {'name': table_name, 'columns': columns, 'workspace': workspace["id"]}
headers = { "X-NI-API-KEY": api_key, 'Content-Type': 'application/json' }
create_table_resp = requests.post(f'{sl_uri}/nidataframe/v1/tables', headers=headers, json=table_info)

# Expect 201 on success, raise an exception if there is an error response
create_table_resp.raise_for_status()

table_id = create_table_resp.json()["id"]
table_id

In [ ]:
# Convert the pandas dataframe to the expected json format
frame_dict = dfErrOut2.reset_index().astype(str).to_json(orient="split", index=False) #.astype(str)

# Write the data to the table. Since we're writing one file/frame per table, set endofData to true
frame_info = f'{{"frame": {frame_dict}, "endOfData": false}}'
headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}
write_data_resp = requests.post(f'{sl_uri}/nidataframe/v1/tables/{table_id}/data', headers=headers, data=frame_info)

# Expect 204 on success, raise an exception if there is an error response
write_data_resp.raise_for_status()



### Write pivoted pandas dataframe into SystemLink DataFrame 

In [ ]:
workspace_id=workspace["id"]
CreateAndUpdateSLDataFrame(dfErrOut2_pivot,workspace_id,table_name='AnomalyScore_pivot')

In [ ]:
dfErrOut2_pivot.to_parquet(r'./dfErrOut2_pivot.parquet')

## Verify: Query Anomaly Score Data Table

In [ ]:
headers = {'X-NI-API-KEY': api_key, 'Content-Type': 'application/json'}
json_body  = {
  "orderBy": [
    {
      "column": "StartTime",
      "descending": True
    }
  ],
  "filters": [
    {
      "column": "Condition",
      "operation": "EQUALS",
      "value": "0"
    },
    {
      "column": "StartTime",
      "operation": "GREATER_THAN_EQUALS",
      "value": "2023-11-01T15:59:14.000Z"
    }
  ],
  "take": 500
}


response=requests.post(f'{sl_uri}/nidataframe/v1/tables/{table_id}/query-data', headers=headers, data=json.dumps(json_body))
if response.status_code == 200 or response.status_code == 201:
    #print("Okay")
    totalRowCount=json.loads(response.text)['totalRowCount']
    continuationToken=json.loads(response.text)['continuationToken']
    print(f"Succesfull Query: Total Rows={totalRowCount}, continuationToken={continuationToken}")
    #print("Response Body: " + json.dumps(json.loads(response.text), indent=4))
if response.status_code >= 400:
    print("Error Response: " + json.dumps(json.loads(response.text), indent=4))

In [ ]:
for i in np.arange(100):
    if continuationToken is not None: 
        json_body.update({"continuationToken":continuationToken})
    response=requests.post(f'{sl_uri}/nidataframe/v1/tables/{table_id}/query-data', headers=headers, data=json.dumps(json_body))
    if response.status_code == 200 or response.status_code == 201:
        #print("Okay")
        totalRowCount=json.loads(response.text)['totalRowCount']
        continuationToken=json.loads(response.text)['continuationToken']
        print(f"Succesfull Query: Total Rows={totalRowCount}, continuationToken={continuationToken}")
        #print("Response Body: " + json.dumps(json.loads(response.text), indent=4))
    if continuationToken is None: break
        

In [ ]:
response.raise_for_status()

In [ ]:
json_body={
  "orderBy": [
    {
      "column": "StartTime",
      "descending": True
    }
  ],
  "filters": [
    {
      "column": "Condition",
      "operation": "EQUALS",
      "value": "0"
    },
    {
      "column": "StartTime",
      "operation": "GREATER_THAN_EQUALS",
      "value": "2023-11-01T15:59:14.000Z"
    }
  ],
  "take": 500
}
continuationToken=None
ldfOut=[]
while(True):
    dfOut, continuationToken, totalRowCount=QueryDataFrame(table_id,sl_uri,json_body=json_body,continuationToken=continuationToken)
    ldfOut.append(dfOut)
    if continuationToken is None: break
    

In [ ]:
dfOutAll=pd.concat(ldfOut)